In [ ]:
# Import library
import numpy as np
import sys
import os
import re
import shutil
import copy
import itertools

In [ ]:
# /* Constant */
PATH = os.getcwd()
NEW_KITUNE_PATH = os.path.join(PATH, "01-新きつね")
QH_PATH = os.path.join(PATH, "02-QH")
OTHER_PATH = os.path.join(PATH, "03-Other")
OUTPUT_PATH = os.path.join(PATH, "10-result")
YMM4 = "-YMM4"

# Array
Y_FOLDER_ARR = np.array(
    ["顔色", "後", "口", "他", "体", "髪", "眉", "目"], 
    dtype=object
)
# [a: close, b, c: open] -> 0: 正順: 口
# [a: open, b, c: close] -> 1: 逆順: 目
NORMAL_ARR = np.array(
    ["後", "口", "他", "他-ピッケル", "他-メガネ", "他-看板", "他-武器シリーズ", "他-兵科記号"], dtype=object
)
NEW_KITUNE_NORMAL_ARR = np.array(
    [os.path.join(name, num) for name in NORMAL_ARR for num in ["00", "01"]], dtype=object
)
NORMAL_ARR = np.append(NORMAL_ARR, NEW_KITUNE_NORMAL_ARR)
REVERSE_ARR = np.array(
    ["目", "目-どんぐり目", "目-丸目", "目-瞳のある目"], dtype=object
)
NEW_KITUNE_REVERSE_ARR = np.array(
    [os.path.join(name, num) for name in REVERSE_ARR for num in ["00", "01"]], dtype=object
)
REVERSE_ARR = np.append(REVERSE_ARR, NEW_KITUNE_REVERSE_ARR)
IGNORE_ARR = np.array(
    ["顔", "全", "体", "体の差分", "髪", "髪の差分", "眉", "服下", "服上"], dtype=object
)
NEW_KITUNE_IGNORE_ARR = np.array(
    [os.path.join(name, num) for name in IGNORE_ARR for num in ["00", "01"]], dtype=object
)
IGNORE_ARR = np.append(IGNORE_ARR, NEW_KITUNE_IGNORE_ARR)

# Dictionary
NEW_KITUNE_DICT = {
    os.path.join("顔", "00"): "顔色",
    os.path.join("顔", "01"): "顔色",
    os.path.join("後", "00"): "後",
    os.path.join("後", "01"): "後",
    os.path.join("口", "00"): "口",
    os.path.join("口", "01"): "口",
    "全": "体",
    os.path.join("他", "00"): "他",
    os.path.join("他", "01"): "他",
    "体": "体",
    os.path.join("髪", "00"): "髪",
    os.path.join("髪", "01"): "髪",
    os.path.join("眉", "00"): "眉",
    os.path.join("眉", "01"): "眉",
    os.path.join("服下", "00"): "後",
    os.path.join("服下", "01"): "後",
    os.path.join("服上", "00"): "他",
    os.path.join("服上", "01"): "他",
    os.path.join("目", "00"): "目",
    os.path.join("目", "01"): "目",
}
QH_DICT = {
    "顔": "顔色",
    "後": "後",
    "口": "口",
    "全": "体",
    "他": "他",
    "他-ピッケル": "他",
    "他-メガネ": "他",
    "他-看板": "他",
    "他-武器シリーズ": "他",
    "他-兵科記号": "他",
    "体": "体",
    "体の差分": "体",
    "髪": "髪",
    "髪の差分": "髪",
    "服下": "後",
    "服上": "他",
    "眉": "眉",
    "目": "目",
    "目-どんぐり目": "目",
    "目-丸目": "目",
    "目-瞳のある目": "目",
}
OTHER_DICT = {
    "顔": "顔色",
    "後": "後",
    "口": "口",
    "全": "体",
    "他": "他",
    "体": "体",
    "髪": "髪",
    "眉": "眉",
    "服下": "後",
    "服上": "他",
    "目": "目",
}

In [ ]:
# /* Function */


In [ ]:
# /* Class */
class Converter:
    def __init__(self):
        self.folder_paths = np.array([], dtype=object)
        self.style_arr = np.array([], dtype=object)
        self.load_paths = np.array([NEW_KITUNE_PATH, QH_PATH, OTHER_PATH], dtype=object)
        self.y_names = np.array([], dtype=object)
        self.output_path_arr = np.array([], dtype=object)
    
    def convert(self):
        print("改名を開始します．")
        self.load_folder()
        self.create_folder()
        self.convert_name()
        print("改名完了．")
    
    def load_folder(self):
        # Get a list of file names only
        for path in self.load_paths:
            files = os.listdir(path)
            files_file = [os.path.join(path, f) for f in files if os.path.isdir(os.path.join(path, f))]
            files_file = np.array(files_file, dtype=object)
            self.folder_paths = np.append(self.folder_paths, files_file)
            style = np.array([path]*len(files_file), dtype=object)
            self.style_arr = np.append(self.style_arr, style)
        return self.folder_paths
    
    def create_folder(self):
        y_names = [os.path.splitext(os.path.basename(p))[0] for p in self.folder_paths]
        y_names = np.array(y_names, dtype=object)
        self.y_names = y_names
        for folder_name, y_path, style in zip(y_names, self.folder_paths, self.style_arr):
            output_path = os.path.join(OUTPUT_PATH, folder_name)
            output_path = output_path + YMM4
            if os.path.isdir(output_path):
                shutil.rmtree(output_path) # ディレクトリを中身ごと削除
            if style != QH_PATH:
                shutil.copytree(
                    y_path, output_path, 
                    ignore=shutil.ignore_patterns(*OTHER_DICT.keys()), dirs_exist_ok=True
                )
            elif style == QH_PATH:
                shutil.copytree(
                    y_path, output_path, 
                    ignore=shutil.ignore_patterns(*QH_DICT.keys()), dirs_exist_ok=True
                )
            for sub_name in Y_FOLDER_ARR:
                sub_path = os.path.join(output_path, sub_name)
                os.makedirs(sub_path, exist_ok=True)
                self.output_path_arr = np.append(self.output_path_arr, sub_path)
        return self.output_path_arr
    
    def convert_name(self):
        for y_path, style in zip(self.folder_paths, self.style_arr):
            y_name = os.path.splitext(os.path.basename(y_path))[0]
            print(str(y_name)+" の改名中……")
            # 辞書の選択
            if style == NEW_KITUNE_PATH:
                y_dict = copy.copy(NEW_KITUNE_DICT)
            elif style == QH_PATH:
                y_dict = copy.copy(QH_DICT)
            elif style == OTHER_PATH:
                y_dict = copy.copy(OTHER_DICT)
            # ゆっくりパーツの改名
            for (n, folder_name) in enumerate( y_dict.keys() ):
                target_path = os.path.join(y_path, folder_name)
                output_path = os.path.join(OUTPUT_PATH, y_name)
                output_path = output_path + YMM4
                output_path = os.path.join(output_path, y_dict[folder_name])
                # コピー元のフォルダーが無ければ、コンテニュー
                if not os.path.isdir(target_path):
                    continue
                # folder_name が IGNORE_ARR に含まれているなら、コピー＆コンテニュー
                if folder_name in IGNORE_ARR:
                    if style != NEW_KITUNE_PATH:
                        shutil.copytree(
                            target_path, output_path, dirs_exist_ok=True
                        )
                    elif style == NEW_KITUNE_PATH:
                        # ファイル名読込
                        files = os.listdir(target_path)
                        files = np.array(files, dtype=object)
                        # コピー
                        for file in files:
                            before_path = os.path.join(target_path, file)
                            rename = str(n).zfill(2)+"-"+file # 改変後の名前
                            after_path = os.path.join(output_path, rename)
                            shutil.copy2(before_path, after_path) # Copy file
                    continue
                # ファイル名読込
                files = os.listdir(target_path)
                files = np.array(files, dtype=object)
                # 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
                num_arr = np.array([], dtype=object)
                for file in files:
                    num_str = re.findall(r'^\d+', file)
                    num_arr = np.append(num_str, num_arr)
                num_arr = np.unique(num_arr) # 重複した要素の削除
                # リネーム
                for target_num in num_arr:
                    # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
                    convert_list = [ s for s in files if (target_num in re.findall(r'^\d+', s)) ]
                    convert_arr = np.array(convert_list, dtype=object)
                    # 順番入れ替え
                    if folder_name in NORMAL_ARR:
                        convert_arr = np.hstack( (convert_arr[-1], convert_arr[:-1]) )
                    elif folder_name in REVERSE_ARR:
                        c_arr = np.flip(convert_arr[1:])
                        convert_arr = np.hstack( (convert_arr[0], c_arr) )
                    # YMM4 Ver. に対応
                    for i, file in enumerate(convert_arr):
                        ext = os.path.splitext(file)[1] # 拡張子抽出
                        before_path = os.path.join(target_path, file)
                        if i == 0:
                            rename = str(n).zfill(2)+"-"+target_num+ext # 改変後の名前
                            after_path = os.path.join(output_path, rename)
                        elif i != 0:
                            rename = str(n).zfill(2)+"-"+target_num+"."+str(i-1)+ext # 改変後の名前
                            after_path = os.path.join(output_path, rename)
                        shutil.copy2(before_path, after_path) # Copy file

In [ ]:
if __name__ == '__main__':
    converter = Converter()
    converter.convert()

In [ ]:
"fafafa口" in "口"

In [ ]:
"口" in "fafafa口"

In [ ]:
"口\\00" in NEW_KITUNE_FOLDER_01_ARR

In [ ]:
"10".zfill(2)

In [ ]:
re.sub(r"\D", "", s)

In [ ]:
files = os.listdir(r"D:\GitHub-Home\00-JupyterHome\2021\ymm4-name-converter\03-Other\00-きつね式せいじゃ\顔")
files = np.array(files, dtype=object)
# 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
num_arr = np.array([], dtype=object)
for file in files:
    num_str = re.findall(r'^\d+', file)
    num_arr = np.append(num_str, num_arr)
num_arr = np.unique(num_arr) # 重複した要素の削除
for target_num in num_arr:
    # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
    convert_list = [ s for s in files if (target_num in re.findall(r'^\d+', s)) ]
convert_list

In [ ]:
num_arr[0] in re.findall(r'^\d+', files[0])

In [ ]:
# /* Constant */
AUTHENTIC_ORDER = 0
REVERSE_ORDER = 1

# Yukkuri Type
ORIGINAL            = "元祖　　　　: Original"
KITUNE              = "きつね式　　: Kitune Type"
NEW_KITUNE          = "新きつね式　: New Kitune Type"
FULL_BODY_KITUNE    = "全身きつね式: Full body Kitune Type"
KAMUI               = "神威式　　　: Kamui Type"
RAKUGAKI            = "らくがき式　: Rakugaki Type"
WATASHI             = "私式。　　　: Watashi Type"
PUNI                = "ぷに式　　　: Puni Type"
QH                  = "QH式　　　　: QH Type"
NININON             = "弐ヶのん式　: Nininon Type"
TORIMISO            = "とりみそ式　: ToriMiso Type"
MIKAN               = "みかん式　　: Mikan Type"
KURO                = "クロ式　　　: Kuro Type"
HACHI               = "蜂旧式　　　: Hachi Type"

# Vector [eye, mouth]
# [a: close, b, c: open] -> 0: 正順: 口
# [a: open, b, c: close] -> 1: 逆順: 目
ORIGINAL_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KITUNE_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
NEW_KITUNE_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8) # special
FULL_BODY_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KAMUI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
RAKUGAKI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
WATASHI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
PUNI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
QH_VECTOR = np.array([
    REVERSE_ORDER, AUTHENTIC_ORDER, 
    REVERSE_ORDER, REVERSE_ORDER, 
    REVERSE_ORDER
], dtype=np.uint8) # special
NININON_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
TORIMISO_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
MIKAN_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KURO_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
HACHI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)

# List
FOLDER_LIST = np.array(["目", "口"], dtype=object)
FOLDER_LIST_NEW_KITUNE = np.array(
    [os.path.join(name, num) for name in FOLDER_LIST for num in ["00", "01"]], dtype=object
)
FOLDER_LIST_QH = np.array(["目", "口", "目-どんぐり目", "目-丸目", "目-瞳のある目"], dtype=object)
Y_TYPE_LIST = np.array([
    ORIGINAL,
    KITUNE,
    NEW_KITUNE,
    FULL_BODY_KITUNE,
    KAMUI,
    RAKUGAKI,
    WATASHI,
    PUNI,
    QH, 
    NININON, 
    TORIMISO, 
    MIKAN, 
    KURO, 
    HACHI
], dtype=object)

# Dictionary
Y_VECTOR_DICT = {
    KAMUI: KAMUI_VECTOR,
    RAKUGAKI: RAKUGAKI_VECTOR,
    WATASHI: WATASHI_VECTOR, 
    NEW_KITUNE: NEW_KITUNE_VECTOR, 
    PUNI: PUNI_VECTOR,
    KITUNE: KITUNE_VECTOR,
    FULL_BODY_KITUNE: FULL_BODY_KITUNE,
    ORIGINAL:ORIGINAL_VECTOR, 
    QH: QH_VECTOR, 
    NININON: NININON_VECTOR, 
    TORIMISO: TORIMISO_VECTOR, 
    MIKAN: MIKAN_VECTOR, 
    KURO: KURO_VECTOR, 
    HACHI: HACHI_VECTOR
}

In [ ]:
# /* Class */
class Converter:
    def __init__(self, path, name, vector_arr, type):
        self.type = type
        self.path = path
        self.name = name
        self.vector_arr = vector_arr
    
    def folder_copy(self):
        if self.type == NEW_KITUNE:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST_NEW_KITUNE
            )
        elif self.type == QH:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST_QH
            )
        else:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST
            )
    
    def folder_copy_main(self, path, name, folder_list):
        folder_path = os.path.join(path, name)
        new_folder_path = os.path.join(path, name+"-YMM4")
        if os.path.isdir(new_folder_path):
            shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
        shutil.copytree(
            folder_path, new_folder_path, 
            ignore=shutil.ignore_patterns(*folder_list), dirs_exist_ok=True
        )
    
    def convert(self):
        print("改名を開始します．\nStart renaming.")
        if self.type == NEW_KITUNE:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST_NEW_KITUNE
            )
        elif self.type == QH:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST_QH
            )
        else:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST
            )
        print("改名完了\nRenaming completed.")
    
    def convert_name(self, path, name, vector_arr, folder_list):
        original_name = copy.copy(name)
        for folder_name, vector in zip(folder_list, vector_arr):
            target_path = os.path.join(path, folder_name)
            new_folder_path = target_path.replace(original_name, original_name+"-YMM4")
            # 改変後のフォルダー作成
            if os.path.isdir(new_folder_path):
                shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
            elif not os.path.isdir(new_folder_path):
                os.makedirs(new_folder_path) # ディレクトリの作成
            # ファイル名読込
            files = os.listdir(target_path)
            files = np.array(files, dtype=object)
            # 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
            num_arr = np.array([], dtype=object)
            for file in files:
                num_str = re.sub(r"\D", "", file)
                num_arr = np.append(num_str, num_arr)
            num_arr = np.unique(num_arr) # 重複した要素の削除
            # リネーム
            for target_num in num_arr:
                # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
                convert_list = [ s for s in files if re.sub(r"\D", "", s)==target_num ]
                convert_arr = np.array(convert_list, dtype=object)
                # 順番入れ替え
                if vector == AUTHENTIC_ORDER:
                    convert_arr = np.hstack( (convert_arr[-1], convert_arr[:-1]) )
                elif vector == REVERSE_ORDER:
                    c_arr = np.flip(convert_arr[1:])
                    convert_arr = np.hstack( (convert_arr[0], c_arr) )
                for i, file in enumerate(convert_arr):
                    ext = os.path.splitext(file)[1] # 拡張子抽出
                    before_path = os.path.join(target_path, file)
                    if i == 0:
                        rename = target_num+ext # 改変後の名前
                        after_path = os.path.join(new_folder_path, rename)
                    elif i != 0:
                        rename = target_num+"."+str(i-1)+ext # 改変後の名前
                        after_path = os.path.join(new_folder_path, rename)
                    shutil.copy2(before_path, after_path) # Copy file

In [ ]:
# /* Function */
# Do not use.
def get_dir_path(relative_path):
    try:
        base_path = sys._MEIPASS
        #print("[Base Path (get from sys)]" + base_path)
    except Exception:
        base_path = os.path.dirname("__file__")
        #print("[Base Path (get from sys)]" + base_path)
    return os.path.join(base_path, relative_path)

def get_folder_name(path):
    files = os.listdir(path)
    files_dir = [f for f in files if os.path.isdir(os.path.join(path, f))]
    files_dir = np.array(files_dir, dtype=object)
    return files_dir

def select_folder(folders):
    print("YMM4 に対応させるフォルダ名を選んでください．")
    print("Select the folder name that corresponds to YMM4.")
    for i in range(len(folders)):
        print("0"+str(i)+": "+folders[i])
    folder_str = input("\n数字を入力して下さい．\nPlease enter a number.\nEx. 1 or 01: ")
    is_number(string=folder_str)
    folder_n = int(folder_str)
    is_corresponding_number(target=folder_n, search_arr=folders)
    print("["+folders[folder_n]+"] を選択しました．\n")
    
    return folders[folder_n]

def is_number(string):
    if( not string.isdigit() ):
        print("入力に文字が検出されました．")
        print("数字のみを入力してください．")
        print("A character was detected in the input.")
        print("Please enter numbers only.")
        
        input("\nPress any key to exit.")
        
        sys.exit()

def is_corresponding_number(target, search_arr):
    if ( not target in range(len(search_arr))):
        print("対応されていない数字が入力されました．")
        print("対応する数字を入力してください．")
        print("An unsupported number has been entered.")
        print("Please enter the corresponding number.")
        
        input("\nPress any key to exit.")
        
        sys.exit()

# Do not use.
def select_direction():
    print("目や口が開いている方向を選んでください．")
    print("Choose the direction in which the eyes and mouth are open.")
    print("\nEx. [a: close, b, c: open] -> 0: 正順")
    print("    [a: open, b, c: close] -> 1: 逆順\n")
    # 目
    eye_n = input("目の数字を入力して下さい．")
    is_number(string=eye_n)
    eye_n = int(eye_n)
    is_corresponding_number(target=eye_n, search_arr=np.arange(2))
    if eye_n == AUTHENTIC_ORDER:
        print("目は [0: 正順] で改名します．")
    elif eye_n == REVERSE_ORDER:
        print("目は [1: 逆順] で改名します．")
    # 口
    mouth_n = input("口の数字を入力して下さい．")
    is_number(string=mouth_n)
    mouth_n = int(mouth_n)
    is_corresponding_number(target=mouth_n, search_arr=np.arange(2))
    if mouth_n == AUTHENTIC_ORDER:
        print("口は [0: 正順] で改名します．")
    elif mouth_n == REVERSE_ORDER:
        print("口は [1: 逆順] で改名します．")
    
    dir_arr = np.array([eye_n, mouth_n], dtype=uint8)
    
    return dir_arr

def convert_name(path, name, vector_arr):
    print("改名を開始します．\nStart renaming.")
    original_name = copy.copy(name)
    for folder_name, vector in zip(FOLDER_LIST, vector_arr):
        target_path = os.path.join(path, folder_name)
        new_folder_path = target_path.replace(original_name, original_name+"-YMM4")
        print("original: "+original_name)
        print(new_folder_path)
        # 改変後のフォルダー作成
        if os.path.isdir(new_folder_path):
            shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
        elif not os.path.isdir(new_folder_path):
            os.mkdir(new_folder_path) # ディレクトリの作成
        # ファイル名読込
        files = os.listdir(target_path)
        #files_file = [f for f in files if os.path.isfile(os.path.join(path, f))]
        files = np.array(files, dtype=object)
        #print(files)
        # 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
        num_arr = np.array([], dtype=object)
        for file in files:
            num_str = re.sub(r"\D", "", file)
            num_arr = np.append(num_str, num_arr)
        num_arr = np.unique(num_arr) # 重複した要素の削除
        #print(num_arr)
        # リネーム
        for target_num in num_arr:
            # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
            convert_list = [ s for s in files if re.sub(r"\D", "", s)==target_num ]
            convert_arr = np.array(convert_list, dtype=object)
            # 単一のファイルならブレイク
            #if len(convert_arr) == 1:
            #    break
            # 順番入れ替え
            if vector == AUTHENTIC_ORDER:
                convert_arr = np.hstack( (convert_arr[-1], convert_arr[:-1]) )
            elif vector == REVERSE_ORDER:
                c_arr = np.flip(convert_arr[1:])
                convert_arr = np.hstack( (convert_arr[0], c_arr) )
            for i, file in enumerate(convert_arr):
                ext = os.path.splitext(file)[1] # 拡張子抽出
                before_path = os.path.join(target_path, file)
                if i == 0:
                    rename = target_num+ext # 改変後の名前
                    after_path = os.path.join(new_folder_path, rename)
                elif i != 0:
                    rename = target_num+"."+str(i-1)+ext # 改変後の名前
                    after_path = os.path.join(new_folder_path, rename)
                
                #print("before: "+before_path)
                #print("after : "+after_path)
                #os.rename(before_path, after_path)
                shutil.copy2(before_path, after_path)
    print("改名完了\nRenaming completed.")

def select_type():
    print("改名するゆっくりのタイプを選択してください．")
    print("Select the Yukkuri type of renaming.\n")
    for i, yukkuri in enumerate(Y_TYPE_LIST):
        print("0"+str(i)+": "+yukkuri)
    y_str = input("\n数字を入力して下さい．\nPlease enter a number.\nEx. 1 or 01: ")
    is_number(string=y_str)
    y_num = int(y_str)
    is_corresponding_number(target=y_num, search_arr=Y_TYPE_LIST)
    type_result = Y_TYPE_LIST[y_num].replace("　", "")
    print("["+type_result+"] を選択しました．\n")
    vector = Y_VECTOR_DICT[Y_TYPE_LIST[y_num]]
    return (vector, Y_TYPE_LIST[y_num])

# Do not use.
def folder_copy(path, name):
    folder_path = os.path.join(path, name)
    new_folder_path = os.path.join(path, name+"-YMM4")
    if os.path.isdir(new_folder_path):
        shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
    shutil.copytree(
        folder_path, new_folder_path, 
        ignore=shutil.ignore_patterns(FOLDER_LIST[0], FOLDER_LIST[1]), dirs_exist_ok=True
    )

def main():
    # 現在のディレクトリ取得
    cur_path = os.getcwd()
    print("Current directory: "+cur_path+"\n")
    # 改名するフォルダの選択
    folders = get_folder_name(path=cur_path)
    folder_name = select_folder(folders=folders)
    folder_path = os.path.join(cur_path, folder_name)
    # ゆっくりのタイプ選択
    vector_arr, y_type = select_type()
    # 改名処理
    #dir_arr = select_direction()
    converter = Converter(type=y_type, path=folder_path, name=folder_name, vector_arr=vector_arr)
    converter.folder_copy()
    #folder_copy(path=cur_path, name=folder_name)
    #convert_name(path=folder_path, name=folder_name, vector_arr=vector_arr)
    converter.convert()

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
input("数字を入力して下さい　Ex. 1 or 01")

In [ ]:
np.arange(2)

In [ ]:
sample_text = np.array(["01a.png", "01b.png", "01c.png", "02.png"], dtype=np.object)

regex = re.compile('d+')

for line in sample_text.splitlines():
  match = regex.findall(line)
  print(match)

In [ ]:
num_arr = np.array(["01a.png", "01b.png", "01c.png", "02.png"], dtype=np.object)
#[ s for s in num_arr if re.sub(r"\D", "", s)=="01" ]
num_arr = np.flip(num_arr)
num_arr

In [ ]:
os.path.splitext(num_arr[0])[1]

In [ ]:
np.arange(2)

In [ ]:
num_arr[:-1]

In [ ]:
np.hstack((num_arr[-1], num_arr[:-1]))

In [ ]:
a = "07a-15"
re.sub(r"\D{2}", "", a)

In [ ]:
os.listdir(r"D:\GitHub-Home\00-JupyterHome\2021\Y3toY4-name-converter\きつね式せいじゃ\目")

In [ ]:
"[きつね式　　：Kitune Type]".replace("　", "")

In [ ]:
folder_copy(path=os.getcwd(), name="aaa")

In [ ]:
python --version

In [ ]:
FOLDER_LIST.tolist()

In [ ]:
a = np.arange(4)
a

In [ ]:
a[1:]

In [ ]:
os.path.join("a", "ppp")

In [ ]:
[os.path.join(p_up, p_down) for p_up in ["目", "口"] for p_down in ["aaa", "bbb"]]

In [ ]:
aa, bb = (0, 1)

In [ ]:
print(aa, bb)

In [ ]:
*[os.path.join(p_up, p_down) for p_up in ["目", "口"] for p_down in ["aaa", "bbb"]]